In [ ]:
from pyspark.sql import SparkSession
from configparser import ConfigParser
from datetime import datetime, timedelta
from logging import INFO, info, warn, error, basicConfig
from os import getcwd, mkdir, remove
from os.path import isdir, isfile
from json import load
from pyspark.sql.functions import *
from pyspark.sql import Window
import time

In [ ]:
session = SparkSession \
        .builder \
        .enableHiveSupport() \
        .appName("FDR CURATED LAYER")\
        .getOrCreate()

In [ ]:
from pyspark.sql import SparkSession, DataFrame
from functools import reduce
from json import load
from boto3 import client
from os.path import basename
from datetime import datetime, timedelta
from pyspark.sql.functions import *



class GetDataFrame:

    def __init__(self):
        self.session = SparkSession\
            .builder \
            .enableHiveSupport() \
            .appName(self.__class__.__name__) \
            .getOrCreate()
        
    def unionAll(self,*dfs):
        return reduce(DataFrame.unionAll, dfs) # will be used to join all the dfs in the dict

    def getDataFrame(self, tableName, days,startdate):
        """
        Returns dataframe of the table in the parameter.
        Needs the number of days for which it needs to pick up the files from s3.
        """

        s3files = [] # This list store the files in the s3 with the prefix.
        conn = client('s3')
        for key in conn.list_objects(Bucket=RAWBUCKET,
                                    Prefix='{}{}/production/1/'\
                                     .format(RAWPREFIX,
                                     tableName))['Contents']:
            s3files.append(basename(key['Key']))
        
        df_dict = {} # used to store the mapping for files with dataframes
        for day in range(days):
            csv = ((datetime.now() - timedelta(startdate)) - timedelta(days=day)).strftime('%Y%m%d')
            for file in s3files:
                if file.startswith(csv):
                    df_dict[csv] = self.session.read.format('com.databricks.spark.csv')\
                        .options(delimiter='\u0001', header=True)\
                        .load("s3n://{}/{}/{}/production/1/{}"\
                              .format(RAWBUCKET, 
                                      RAWPREFIX,
                                      tableName,
                                      file))

        return self.unionAll(*df_dict.values())

In [ ]:
RAWBUCKET = 'fusion-pdx-140984961647-global-ecommerce-newgistics'
RAWPREFIX = 's/d/r/fdr/ngsdw/'

In [ ]:
getDataFrameInstance = GetDataFrame()

### Loading Data

In [ ]:
trackinglabel_raw_df = session.read.format('com.databricks.spark.csv')\
                        .options(delimiter='\u0001', header=True)\
                        .load("s3n://fusion-pdx-140984961647-global-ecommerce-newgistics/s/d/r/fdr/ngsdw/trackinglabel/production/1/")

In [ ]:
package_raw_df = session.read.format('com.databricks.spark.csv')\
                        .options(delimiter='\u0001', header=True)\
                        .load("s3n://fusion-pdx-140984961647-global-ecommerce-newgistics/s/d/r/fdr/ngsdw/package/production/1/")

In [ ]:
trackinglabelevent_raw_df = session.read.format('com.databricks.spark.csv')\
                        .options(delimiter='\u0001', header=True)\
                        .load("s3n://fusion-pdx-140984961647-global-ecommerce-newgistics/s/d/r/fdr/ngsdw/trackinglabelevent/production/1/")

In [ ]:
timedim = session.read.format('com.databricks.spark.csv')\
                .options(delimiter='\u0001', header=True)\
                .load("s3n://fusion-pdx-140984961647-global-ecommerce-newgistics/s/d/r/fdr/ngsdw/timedim/production/1/")

In [ ]:
facilitydim = session.read.format('com.databricks.spark.csv')\
                .options(delimiter='\u0001', header=True)\
                .load("s3n://fusion-pdx-140984961647-global-ecommerce-newgistics/s/d/r/fdr/ngsdw/facilitydim/production/1/")

In [ ]:
trackingeventdim = session.read.format('com.databricks.spark.csv')\
                .options(delimiter='\u0001', header=True)\
                .load("s3n://fusion-pdx-140984961647-global-ecommerce-newgistics/s/d/r/fdr/ngsdw/trackingeventdim/production/1/")

In [ ]:
orgdim_df = session.read.format('com.databricks.spark.csv')\
                .options(delimiter='\u0001', header=True)\
                .load("s3n://fusion-pdx-140984961647-global-ecommerce-newgistics/s/d/r/fdr/ngsdw/orgdim/production/1/")

### Applying Basic Filteration

In [ ]:
trackinglabel_df = trackinglabel_raw_df\
.filter(to_date('CREATEDATE') >= '2020-04-18').distinct()

In [ ]:
package_df = package_raw_df\
.filter(to_date(from_unixtime(unix_timestamp(col('CREATEDATEKEY'), 'yyyyMMdd'))) >= '2020-04-18').distinct()

In [ ]:
trackinglabelevent_df = trackinglabelevent_raw_df\
.filter(to_date('CREATEDATE') >= '2020-04-18').distinct()


#### ========================================= Checkpoint 1 =========================================

In [ ]:
trackinglabel_df.createOrReplaceTempView('trackinglabel_df')

# session.sql('drop table hive_trackinglabel_df')

session.sql('create table hive_trackinglabel_df as select * from trackinglabel_df')

In [ ]:
package_df.createOrReplaceTempView('package_df')

# session.sql('drop table hive_package_df')

session.sql('create table hive_package_df as select * from package_df')

In [ ]:
trackinglabelevent_df.createOrReplaceTempView('trackinglabelevent_df')

# session.sql('drop table hive_trackinglabelevent_df')

session.sql('create table hive_trackinglabelevent_df as select * from trackinglabelevent_df')

In [ ]:
trackinglabel_df = session.sql('select * from hive_trackinglabel_df')

In [ ]:
package_df = session.sql('select * from hive_package_df')

In [ ]:
trackinglabelevent_df = session.sql('select * from hive_trackinglabelevent_df')

#### ========================================= Checkpoint 1 ENDS =========================================

### Filteration | Count

In [ ]:
trackinglabel_df.select('TRACKINGLABELKEY').distinct().count()

In [ ]:
package_df.select('PACKAGEKEY').distinct().count()

In [ ]:
trackinglabelevent_df.select('TRACKINGLABELKEY').distinct().count()

# Applying Curation

In [ ]:
tlk = Window.partitionBy('TRACKINGLABELKEY')

In [ ]:
curated_trackinglabel_stage1_df=trackinglabel_df\
.withColumn('MAX_SFUPDATETS', max('SFUPDATETS').over(tlk))\
.filter(col('MAX_SFUPDATETS') == col('SFUPDATETS'))\
.select('TRACKINGLABELKEY',
        'PRIMARYBARCODE',
        'PRSBARCODE',
        'MERCHANTORGKEY',
        'CREATEDATEKEY',
        'SORTCODE',
       'DESTFACILITYKEY')\
.withColumn('PRSBARCODEFLAG', col('PRSBARCODE').substr(1, 3))\
.withColumn('PRIMARYBARCODEFLAG', col('PRIMARYBARCODE').substr(1, 3)).distinct()

In [ ]:
curated_trackinglabel_stage2_df=curated_trackinglabel_stage1_df\
.select("PRIMARYBARCODE")\
.groupBy('PRIMARYBARCODE')\
.agg(count('PRIMARYBARCODE').alias('PRIMARYBARCODE_COUNT'))\
.filter(col('PRIMARYBARCODE_COUNT') <= 2).select("PRIMARYBARCODE").distinct()

In [ ]:
curated_trackinglabel_stage3_df=curated_trackinglabel_stage1_df\
.join(curated_trackinglabel_stage2_df,\
      curated_trackinglabel_stage1_df.PRIMARYBARCODE == curated_trackinglabel_stage2_df.PRIMARYBARCODE, "right")\
.select(curated_trackinglabel_stage1_df.TRACKINGLABELKEY,
curated_trackinglabel_stage1_df.PRIMARYBARCODE,
curated_trackinglabel_stage1_df.PRSBARCODE,
curated_trackinglabel_stage1_df.MERCHANTORGKEY,
curated_trackinglabel_stage1_df.CREATEDATEKEY,
curated_trackinglabel_stage1_df.PRSBARCODEFLAG,
curated_trackinglabel_stage1_df.PRIMARYBARCODEFLAG,
curated_trackinglabel_stage1_df.DESTFACILITYKEY,
curated_trackinglabel_stage1_df.SORTCODE).distinct()

### Unique Identifier Creation

In [ ]:
unique_window = Window.partitionBy().orderBy('CREATEDATEKEY','PRIMARYBARCODE','TRACKINGLABELKEY')

In [ ]:
curated_trackinglabel_stage4_df = curated_trackinglabel_stage3_df\
.select(curated_trackinglabel_stage3_df.TRACKINGLABELKEY,
curated_trackinglabel_stage3_df.PRIMARYBARCODE,
curated_trackinglabel_stage3_df.PRSBARCODE,
curated_trackinglabel_stage3_df.MERCHANTORGKEY,
curated_trackinglabel_stage3_df.CREATEDATEKEY,
curated_trackinglabel_stage3_df.PRSBARCODEFLAG,
curated_trackinglabel_stage3_df.PRIMARYBARCODEFLAG,
curated_trackinglabel_stage3_df.DESTFACILITYKEY,
curated_trackinglabel_stage3_df.SORTCODE)\
.withColumn("PREVIOUSVALUE",lag(curated_trackinglabel_stage3_df.CREATEDATEKEY).over(unique_window))

In [ ]:
curated_trackinglabel_stage5_df = curated_trackinglabel_stage4_df\
.select(curated_trackinglabel_stage4_df.TRACKINGLABELKEY,
curated_trackinglabel_stage4_df.PRIMARYBARCODE,
curated_trackinglabel_stage4_df.PRSBARCODE,
curated_trackinglabel_stage4_df.MERCHANTORGKEY,
curated_trackinglabel_stage4_df.CREATEDATEKEY,
curated_trackinglabel_stage4_df.PRSBARCODEFLAG,
curated_trackinglabel_stage4_df.PRIMARYBARCODEFLAG,
curated_trackinglabel_stage4_df.DESTFACILITYKEY,
curated_trackinglabel_stage4_df.SORTCODE,
curated_trackinglabel_stage4_df.PREVIOUSVALUE)\
.withColumn("DATEDIFF", when(((curated_trackinglabel_stage4_df.CREATEDATEKEY\
                              - curated_trackinglabel_stage4_df.PREVIOUSVALUE)<=75)\
                             | (curated_trackinglabel_stage4_df.PREVIOUSVALUE.isNull()), '#1')
                              .otherwise('#2'))

In [ ]:
curated_trackinglabel_df = curated_trackinglabel_stage5_df\
.select(curated_trackinglabel_stage5_df.TRACKINGLABELKEY,
curated_trackinglabel_stage5_df.PRIMARYBARCODE,
curated_trackinglabel_stage5_df.PRSBARCODE,
curated_trackinglabel_stage5_df.MERCHANTORGKEY,
curated_trackinglabel_stage5_df.CREATEDATEKEY,
curated_trackinglabel_stage5_df.PRSBARCODEFLAG,
curated_trackinglabel_stage5_df.PRIMARYBARCODEFLAG,
curated_trackinglabel_stage5_df.DESTFACILITYKEY,
curated_trackinglabel_stage5_df.SORTCODE,
curated_trackinglabel_stage5_df.PREVIOUSVALUE,
curated_trackinglabel_stage5_df.DATEDIFF)\
.withColumn("ID", concat(curated_trackinglabel_stage5_df\
                          .PRIMARYBARCODE,curated_trackinglabel_stage5_df.DATEDIFF))\
.filter(col('TRACKINGLABELKEY').isNotNull())

### Unique Identifier Creation Ends

In [ ]:
pk = Window.partitionBy('PACKAGEKEY')

In [ ]:
curated_package_stage1_df=package_df\
.withColumn('MAX_SFUPDATETS', max('SFUPDATETS').over(pk))\
.filter(col('MAX_SFUPDATETS') == col('SFUPDATETS'))\
.select('PRIMARYBARCODE',
        'PRSBARCODE',
        'MERCHANTORGKEY',
        'SMARTLABELBARCODE',
        'DESTFACILITYKEY',
        'DELIVERYTIMEKEY',
        'DELIVERYDATEKEY',
        'SORTCODE',
        'CREATEDATEKEY',
        'PACKAGEKEY')\
.withColumn('PRSBARCODEFLAG', col('PRSBARCODE').substr(1, 3))\
.withColumn('PRIMARYBARCODEFLAG', col('PRIMARYBARCODE').substr(1, 3)).distinct()

In [ ]:
curated_package_stage2_df=curated_package_stage1_df\
.select('PRSBARCODE')\
.groupBy('PRSBARCODE')\
.agg(count('PRSBARCODE').alias('PRSBARCODE_COUNT'))\
.filter(col('PRSBARCODE_COUNT') == 1).select("PRSBARCODE").distinct()

In [ ]:
curated_package_df=curated_package_stage1_df\
.join(curated_package_stage2_df,\
      curated_package_stage1_df.PRSBARCODE == curated_package_stage2_df.PRSBARCODE, "right")\
.select(curated_package_stage1_df.PRIMARYBARCODE,
curated_package_stage1_df.PRSBARCODE,
curated_package_stage1_df.MERCHANTORGKEY,
curated_package_stage1_df.SMARTLABELBARCODE,
curated_package_stage1_df.DESTFACILITYKEY,
curated_package_stage1_df.DELIVERYTIMEKEY,
curated_package_stage1_df.DELIVERYDATEKEY,
curated_package_stage1_df.SORTCODE,
curated_package_stage1_df.CREATEDATEKEY,
curated_package_stage1_df.PRSBARCODEFLAG,
curated_package_stage1_df.PRIMARYBARCODEFLAG,
curated_package_stage1_df.PACKAGEKEY).distinct()

#### ========================================= Checkpoint 2 =========================================

In [ ]:
curated_trackinglabel_df.createOrReplaceTempView('V_curated_trackinglabel_df')
session.sql('create table hive_curated_trackinglabel_df as select * from V_curated_trackinglabel_df')

In [ ]:
curated_package_df.createOrReplaceTempView('V_curated_package_df')
session.sql('create table hive_curated_package_df as select * from V_curated_package_df')

In [ ]:
curated_trackinglabel_df = session.sql('select * from hive_curated_trackinglabel_df')

In [ ]:
curated_package_df = session.sql('select * from hive_curated_package_df')

#### ========================================= Checkpoint 2 ENDS =========================================

# Curation | Count

In [ ]:
curated_trackinglabel_stage1_df.select('TRACKINGLABELKEY').distinct().count()

In [ ]:
curated_trackinglabel_stage3_df.select('TRACKINGLABELKEY').distinct().count()

In [ ]:
curated_trackinglabel_stage4_df.select('TRACKINGLABELKEY').distinct().count()

In [ ]:
curated_trackinglabel_stage5_df.select('TRACKINGLABELKEY').distinct().count()

In [ ]:
curated_trackinglabel_df.select('TRACKINGLABELKEY').distinct().count()

In [ ]:
curated_trackinglabel_df.count()

In [ ]:
curated_trackinglabel_df.select('ID').distinct().count()

In [ ]:
curated_trackinglabel_df.select('PRIMARYBARCODE').distinct().count()

In [ ]:
curated_trackinglabel_df\
.filter(col('DESTFACILITYKEY') == '0')\
.select('TRACKINGLABELKEY').distinct().count()

In [ ]:
curated_trackinglabel_df\
.filter(col('DESTFACILITYKEY') == '1')\
.select('TRACKINGLABELKEY').distinct().count()

In [ ]:
curated_trackinglabel_df\
.filter(col('DESTFACILITYKEY') == 'N')\
.select('TRACKINGLABELKEY').distinct().count()

In [ ]:
curated_trackinglabel_df\
.filter(col('DESTFACILITYKEY').isNull())\
.select('TRACKINGLABELKEY').distinct().count()

### Mapping with Dimensional Tables

In [ ]:
curated_trackinglabel_orgdim_df = curated_trackinglabel_df\
.join(broadcast(orgdim_df),\
     curated_trackinglabel_df.MERCHANTORGKEY==orgdim_df.ORGKEY,"left")\
.select(curated_trackinglabel_df.TRACKINGLABELKEY,curated_trackinglabel_df.PRSBARCODE,\
              curated_trackinglabel_df.PRIMARYBARCODE,curated_trackinglabel_df.MERCHANTORGKEY,\
              curated_trackinglabel_df.PRSBARCODEFLAG,curated_trackinglabel_df.PRIMARYBARCODEFLAG,\
              curated_trackinglabel_df.CREATEDATEKEY,curated_trackinglabel_df.DESTFACILITYKEY,\
              curated_trackinglabel_df.SORTCODE,curated_trackinglabel_df.ID,orgdim_df.MERCHANTID).distinct()

In [ ]:
curated_package_facilitydim_df = curated_package_df.join(broadcast(facilitydim),\
                                                curated_package_df.DESTFACILITYKEY==facilitydim.FACILITYKEY,"left")\
.join(broadcast(timedim), curated_package_df.DELIVERYTIMEKEY==timedim.TIMEKEY, "left")\
.select(curated_package_df.SMARTLABELBARCODE,curated_package_df.MERCHANTORGKEY,\
              curated_package_df.PRIMARYBARCODE,curated_package_df.PRSBARCODE,\
              curated_package_df.PRSBARCODEFLAG,curated_package_df.PRIMARYBARCODEFLAG,\
              curated_package_df.DESTFACILITYKEY,curated_package_df.CREATEDATEKEY,\
              curated_package_df.DELIVERYDATEKEY,curated_package_df.SORTCODE,curated_package_df.PACKAGEKEY,\
              timedim.TIMEOFDAY.alias('DELIVERYTIMEKEY'),facilitydim.FACILITYNAME.alias("DESTFACILITYNAME")).distinct()

In [ ]:
curated_tracking_label_event_with_dim_df = trackinglabelevent_df\
.join(broadcast(trackingeventdim), ['TRACKINGEVENTKEY'] , "left")\
.join(broadcast(facilitydim), ['FACILITYKEY'], "left")\
.join(broadcast(timedim), trackinglabelevent_df.EVENTRPTTIMEKEY==timedim.TIMEKEY, "left")\
.select(trackingeventdim.DISPLAYTRACKINGEVENT.alias('DISPLAYTRACKINGEVENT'),
        trackingeventdim.EVENTCODE.alias('EVENTCODE'),
        timedim.TIMEOFDAY.alias('EVENTTIME'),
        trackinglabelevent_df.FACILITYKEY.alias('EVENTFACILITYKEY'),
        facilitydim.FACILITYNAME.alias('EVENTFACILITYNAME'),
        trackinglabelevent_df.TRACKINGEVENTKEY.alias('TRACKINGEVENTKEY'),
        trackinglabelevent_df.TRACKINGLABELKEY.alias('TRACKINGLABELKEY'),
        trackinglabelevent_df.EVENTRPTDATEKEY.alias('EVENTDATE')).distinct()

#### ========================================= Checkpoint 3 =========================================

In [ ]:
curated_trackinglabel_orgdim_df.createOrReplaceTempView('V_curated_trackinglabel_orgdim_df')
session.sql('create table hive_curated_trackinglabel_orgdim_df as select * from V_curated_trackinglabel_orgdim_df')

In [ ]:
curated_package_facilitydim_df.createOrReplaceTempView('V_curated_package_facilitydim_df')
session.sql('create table hive_curated_package_facilitydim_df as select * from V_curated_package_facilitydim_df') 

In [ ]:
curated_tracking_label_event_with_dim_df.createOrReplaceTempView('V_curated_tracking_label_event_with_dim_df')
session.sql('create table hive_curated_tracking_label_event_with_dim_df as select * from V_curated_tracking_label_event_with_dim_df')  

In [ ]:
curated_trackinglabel_orgdim_df = session.sql('select * from hive_curated_trackinglabel_orgdim_df')

In [ ]:
curated_package_facilitydim_df = session.sql('select * from hive_curated_package_facilitydim_df')

In [ ]:
curated_tracking_label_event_with_dim_df = session.sql('select * from hive_curated_tracking_label_event_with_dim_df')

#### ========================================= Checkpoint 3 END =========================================

# Dimensional Tables Mapping | Count

In [ ]:
curated_trackinglabel_orgdim_df.select('ID').distinct().count()

In [ ]:
curated_trackinglabel_orgdim_df\
.filter(col('DESTFACILITYKEY') == '0')\
.select('TRACKINGLABELKEY').distinct().count()

In [ ]:
curated_trackinglabel_orgdim_df\
.filter(col('DESTFACILITYKEY') == '1')\
.select('TRACKINGLABELKEY').distinct().count()

In [ ]:
curated_trackinglabel_orgdim_df\
.filter(col('DESTFACILITYKEY') == 'N')\
.select('TRACKINGLABELKEY').distinct().count()

In [ ]:
curated_trackinglabel_orgdim_df\
.filter(col('DESTFACILITYKEY').isNull())\
.select('TRACKINGLABELKEY').distinct().count()

In [ ]:
curated_package_facilitydim_df\
.filter(col('DESTFACILITYKEY') == '1')\
.select('packagekey').distinct().count()

In [ ]:
curated_package_facilitydim_df\
.filter(col('DESTFACILITYKEY') == '0')\
.select('packagekey').distinct().count()

In [ ]:
curated_package_facilitydim_df\
.filter(col('DESTFACILITYKEY') == 'N')\
.select('packagekey').distinct().count()

In [ ]:
curated_package_facilitydim_df\
.filter(col('DESTFACILITYKEY').isNull())\
.select('packagekey').distinct().count()

### Mapping with Transactional table

In [ ]:
unique_prsbarcode_df=curated_trackinglabel_orgdim_df\
.select('PRSBARCODE')\
.groupBy('PRSBARCODE')\
.agg(count('PRSBARCODE').alias('PRSBARCODE_COUNT'))\
.filter(col('PRSBARCODE_COUNT') == 1).select("PRSBARCODE").distinct()

In [ ]:
trackinglabel_unique_prsbarcode_df=curated_trackinglabel_orgdim_df\
.join(unique_prsbarcode_df,\
      curated_trackinglabel_orgdim_df.PRSBARCODE == unique_prsbarcode_df.PRSBARCODE, "right")\
.select(curated_trackinglabel_orgdim_df.TRACKINGLABELKEY,
curated_trackinglabel_orgdim_df.PRSBARCODE,
curated_trackinglabel_orgdim_df.PRIMARYBARCODE,
curated_trackinglabel_orgdim_df.MERCHANTORGKEY,
curated_trackinglabel_orgdim_df.PRSBARCODEFLAG,
curated_trackinglabel_orgdim_df.PRIMARYBARCODEFLAG,
curated_trackinglabel_orgdim_df.CREATEDATEKEY,
curated_trackinglabel_orgdim_df.DESTFACILITYKEY,
curated_trackinglabel_orgdim_df.SORTCODE,
curated_trackinglabel_orgdim_df.MERCHANTID,
curated_trackinglabel_orgdim_df.ID).distinct()

In [ ]:
curated_trackinglabel_package_df = curated_package_facilitydim_df\
      .join(trackinglabel_unique_prsbarcode_df,\
            trackinglabel_unique_prsbarcode_df.PRSBARCODE ==\
            curated_package_facilitydim_df.PRSBARCODE, "right")\
.select(trackinglabel_unique_prsbarcode_df.TRACKINGLABELKEY,
        trackinglabel_unique_prsbarcode_df.PRSBARCODE,
        trackinglabel_unique_prsbarcode_df.PRIMARYBARCODE,
        trackinglabel_unique_prsbarcode_df.MERCHANTID,
        trackinglabel_unique_prsbarcode_df.MERCHANTORGKEY,
        trackinglabel_unique_prsbarcode_df.PRSBARCODEFLAG,
        trackinglabel_unique_prsbarcode_df.PRIMARYBARCODEFLAG,
        trackinglabel_unique_prsbarcode_df.CREATEDATEKEY,
        trackinglabel_unique_prsbarcode_df.ID,
        (when(((trackinglabel_unique_prsbarcode_df.DESTFACILITYKEY == '0')\
               | (trackinglabel_unique_prsbarcode_df.DESTFACILITYKEY == '1')),\
                                   curated_package_facilitydim_df.DESTFACILITYKEY)\
            .otherwise(trackinglabel_unique_prsbarcode_df.DESTFACILITYKEY)).alias('DESTFACILITYKEY'),
        (when((trackinglabel_unique_prsbarcode_df.SORTCODE == 'N'),\
                            curated_package_facilitydim_df.SORTCODE)\
            .otherwise(trackinglabel_unique_prsbarcode_df.SORTCODE)).alias('SORTCODE'),
        curated_package_facilitydim_df.DELIVERYDATEKEY,
        curated_package_facilitydim_df.DELIVERYTIMEKEY,
        curated_package_facilitydim_df.DESTFACILITYNAME,
).distinct()

In [ ]:
unmapped_trackinglabel_df = curated_trackinglabel_orgdim_df\
.select('TRACKINGLABELKEY').distinct()\
.join(curated_trackinglabel_package_df, curated_trackinglabel_orgdim_df.TRACKINGLABELKEY\
      == curated_trackinglabel_package_df.TRACKINGLABELKEY, "leftanti").distinct()

In [ ]:
unmapped_trackinglabel_with_dummy_package_df=curated_trackinglabel_orgdim_df\
.join(broadcast(unmapped_trackinglabel_df),\
      curated_trackinglabel_orgdim_df.TRACKINGLABELKEY == unmapped_trackinglabel_df.TRACKINGLABELKEY, "right")\
.select(curated_trackinglabel_orgdim_df.TRACKINGLABELKEY,
curated_trackinglabel_orgdim_df.PRSBARCODE,
curated_trackinglabel_orgdim_df.PRIMARYBARCODE,
curated_trackinglabel_orgdim_df.MERCHANTID,
curated_trackinglabel_orgdim_df.MERCHANTORGKEY,
curated_trackinglabel_orgdim_df.PRSBARCODEFLAG,
curated_trackinglabel_orgdim_df.PRIMARYBARCODEFLAG,
curated_trackinglabel_orgdim_df.CREATEDATEKEY,
curated_trackinglabel_orgdim_df.ID,
curated_trackinglabel_orgdim_df.DESTFACILITYKEY,
curated_trackinglabel_orgdim_df.SORTCODE)\
.withColumn('DELIVERYDATEKEY', lit(None).cast("string"))\
.withColumn('DELIVERYTIMEKEY', lit(None).cast("string"))\
.withColumn('DESTFACILITYNAME', lit(None).cast("string"))\
.distinct()

In [ ]:
parcel_tracking_df = curated_trackinglabel_package_df.union(unmapped_trackinglabel_with_dummy_package_df)

In [ ]:
total_parcel_events_df=parcel_tracking_df\
.join(curated_tracking_label_event_with_dim_df, curated_tracking_label_event_with_dim_df.TRACKINGLABELKEY\
      == parcel_tracking_df.TRACKINGLABELKEY , "left")\
.select(
parcel_tracking_df['TRACKINGLABELKEY'],
parcel_tracking_df['PRSBARCODE'],
parcel_tracking_df['PRIMARYBARCODE'],
parcel_tracking_df['SORTCODE'],
parcel_tracking_df['MERCHANTORGKEY'],
parcel_tracking_df['DESTFACILITYKEY'],
parcel_tracking_df['DESTFACILITYNAME'],
parcel_tracking_df['CREATEDATEKEY'],
parcel_tracking_df['ID'],
parcel_tracking_df['DELIVERYDATEKEY'],
parcel_tracking_df['DELIVERYTIMEKEY'],
curated_tracking_label_event_with_dim_df['EVENTFACILITYKEY'],
curated_tracking_label_event_with_dim_df['EVENTFACILITYNAME'],
curated_tracking_label_event_with_dim_df['TRACKINGEVENTKEY'],
curated_tracking_label_event_with_dim_df['EVENTCODE'],
curated_tracking_label_event_with_dim_df['DISPLAYTRACKINGEVENT'],
curated_tracking_label_event_with_dim_df['EVENTDATE'],
curated_tracking_label_event_with_dim_df['EVENTTIME']
)\
.withColumn('BOLNUMBER', lit(None).cast("string")).distinct()

#### ========================================= Checkpoint 4 =========================================

In [ ]:
parcel_tracking_df.createOrReplaceTempView('V_parcel_tracking_df')
session.sql('create table hive_parcel_tracking_df as select * from V_parcel_tracking_df')

In [ ]:
total_parcel_events_df.createOrReplaceTempView('V_total_parcel_events_df')
session.sql('create table hive_total_parcel_events_df as select * from V_total_parcel_events_df')

In [ ]:
parcel_tracking_df = session.sql('select * from hive_parcel_tracking_df')

In [ ]:
total_parcel_events_df = session.sql('select * from hive_total_parcel_events_df')

#### ========================================= Checkpoint 4 ENDS =========================================

### Mapping With Event Tables

### Trnsactional table mapping | Count

In [ ]:
unique_prsbarcode_df.count()

In [ ]:
unmapped_trackinglabel_with_dummy_package_df.count()

In [ ]:
unmapped_trackinglabel_with_dummy_package_df.select('TRACKINGLABELKEY').distinct().count()

In [ ]:
unmapped_trackinglabel_df.count()

In [ ]:
curated_trackinglabel_package_df\
.filter(col('DESTFACILITYKEY') == '0')\
.select('TRACKINGLABELKEY').distinct().count()

In [ ]:
curated_trackinglabel_package_df\
.filter(col('DESTFACILITYKEY') == '1')\
.select('TRACKINGLABELKEY').distinct().count()

In [ ]:
curated_trackinglabel_package_df\
.filter(col('DESTFACILITYKEY') == 'N')\
.select('TRACKINGLABELKEY').distinct().count()

In [ ]:
curated_trackinglabel_package_df\
.filter(col('DESTFACILITYKEY').isNull())\
.select('TRACKINGLABELKEY').distinct().count()

In [ ]:
parcel_tracking_df.count()

In [ ]:
parcel_tracking_df.select('TRACKINGLABELKEY').distinct().count()

In [ ]:
parcel_tracking_df\
.filter(col('DESTFACILITYKEY') == '0')\
.select('TRACKINGLABELKEY').distinct().count()

In [ ]:
parcel_tracking_df\
.filter(col('DESTFACILITYKEY') == 'N')\
.select('TRACKINGLABELKEY').distinct().count()

In [ ]:
parcel_tracking_df\
.filter(col('DESTFACILITYKEY') == '1')\
.select('TRACKINGLABELKEY').distinct().count()

In [ ]:
parcel_tracking_df\
.filter(col('DESTFACILITYKEY').isNull())\
.select('TRACKINGLABELKEY').distinct().count()

In [ ]:
parcel_tracking_df\
.filter((col('DELIVERYDATEKEY').isNull()) | (col('DELIVERYDATEKEY') == 'N'))\
.select('TRACKINGLABELKEY').distinct().count()

### Mapping with Transactional table ends

# Transactional Tables Mapping | Count

In [ ]:
total_parcel_events_df.select('TRACKINGLABELKEY').distinct().count()

In [ ]:
total_parcel_events_df\
.filter(col('DESTFACILITYKEY') == '0')\
.select('TRACKINGLABELKEY').distinct().count()

In [ ]:
total_parcel_events_df\
.filter(col('DESTFACILITYKEY') == '1')\
.select('TRACKINGLABELKEY').distinct().count()

In [ ]:
total_parcel_events_df\
.filter(col('DESTFACILITYKEY') == 'N')\
.select('TRACKINGLABELKEY').distinct().count()

In [ ]:
total_parcel_events_df\
.filter(col('DESTFACILITYKEY').isNull())\
.select('TRACKINGLABELKEY').distinct().count()

In [ ]:
total_parcel_events_df\
.filter((col('DELIVERYDATEKEY').isNull()) | (col('DELIVERYDATEKEY') == 'N'))\
.select('TRACKINGLABELKEY').distinct().count()

In [ ]:
total_parcel_events_df.count()

# BACKFILLING MISSING DELIVERY EVENTS FROM LEGACY

In [ ]:
delivery_tracking_label_df = total_parcel_events_df\
.filter(col('TRACKINGEVENTKEY')=='13')\
.select('TRACKINGLABELKEY').distinct()

In [ ]:
missing_delivery_tracking_label_df = parcel_tracking_df\
.filter((col('DELIVERYDATEKEY').isNotNull()) & (col('DELIVERYDATEKEY') != 'N'))\
.select('TRACKINGLABELKEY').distinct()\
.join(delivery_tracking_label_df, parcel_tracking_df.TRACKINGLABELKEY\
      == delivery_tracking_label_df.TRACKINGLABELKEY, "leftanti").distinct()

In [ ]:
missing_delivery_parcel_df=missing_delivery_tracking_label_df\
.select(missing_delivery_tracking_label_df.TRACKINGLABELKEY.alias('TRACKINGLABELKEY_V4'))

In [ ]:
parcel_tracking_v3_df=parcel_tracking_df\
.select(parcel_tracking_df.TRACKINGLABELKEY.alias('TRACKINGLABELKEY_V3'),
parcel_tracking_df.PRSBARCODE.alias('PRSBARCODE_V3'),
parcel_tracking_df.PRIMARYBARCODE.alias('PRIMARYBARCODE_V3'),
parcel_tracking_df.MERCHANTORGKEY.alias('MERCHANTORGKEY_V3'),
parcel_tracking_df.CREATEDATEKEY.alias('CREATEDATEKEY_V3'),
parcel_tracking_df.DELIVERYDATEKEY.alias('DELIVERYDATEKEY_V3'),
parcel_tracking_df.DELIVERYTIMEKEY.alias('DELIVERYTIMEKEY_V3'),
parcel_tracking_df.DESTFACILITYNAME.alias('DESTFACILITYNAME_V3'),
parcel_tracking_df.DESTFACILITYKEY.alias('DESTFACILITYKEY_V3'),
parcel_tracking_df.SORTCODE.alias('SORTCODE_V3'),
parcel_tracking_df.ID.alias('ID_V3'))

In [ ]:
missing_delivery_events_parcel=missing_delivery_parcel_df\
.join(parcel_tracking_v3_df,\
missing_delivery_parcel_df.TRACKINGLABELKEY_V4 == parcel_tracking_v3_df.TRACKINGLABELKEY_V3,"leftouter")\
      .select(missing_delivery_parcel_df.TRACKINGLABELKEY_V4.alias('TRACKINGLABELKEY'),
parcel_tracking_v3_df.PRSBARCODE_V3.alias('PRSBARCODE'),
parcel_tracking_v3_df.PRIMARYBARCODE_V3.alias('PRIMARYBARCODE'),
parcel_tracking_v3_df.MERCHANTORGKEY_V3.alias('MERCHANTORGKEY'),
parcel_tracking_v3_df.SORTCODE_V3.alias('SORTCODE'),
parcel_tracking_v3_df.DESTFACILITYKEY_V3.alias('DESTFACILITYKEY'),
parcel_tracking_v3_df.DELIVERYDATEKEY_V3.alias('DELIVERYDATEKEY'),
parcel_tracking_v3_df.DELIVERYDATEKEY_V3.alias('DELIVERYTIMEKEY'),
parcel_tracking_v3_df.DESTFACILITYNAME_V3.alias('DESTFACILITYNAME'),
parcel_tracking_v3_df.CREATEDATEKEY_V3.alias('CREATEDATEKEY'),
parcel_tracking_v3_df.DELIVERYDATEKEY_V3.alias('EVENTDATE'),
parcel_tracking_v3_df.DELIVERYDATEKEY_V3.alias('EVENTTIME'),
parcel_tracking_v3_df.ID_V3.alias('ID'),
parcel_tracking_v3_df.DESTFACILITYKEY_V3.alias('EVENTFACILITYKEY'),
parcel_tracking_v3_df.DESTFACILITYNAME_V3.alias('EVENTFACILITYNAME'))\
.withColumn('TRACKINGEVENTKEY', lit('13'))\
.withColumn('EVENTCODE', lit('DRC'))\
.withColumn('DISPLAYTRACKINGEVENT', lit(None).cast("string"))\
.withColumn('BOLNUMBER', lit(None).cast("string"))

In [ ]:
missing_parcel_tracking_df=missing_delivery_events_parcel.\
select('ID',
'TRACKINGLABELKEY',
'PRIMARYBARCODE',
'SORTCODE',
'MERCHANTORGKEY',
'DESTFACILITYKEY',
'DESTFACILITYNAME',
'CREATEDATEKEY',
'EVENTFACILITYKEY',
'EVENTFACILITYNAME',
'TRACKINGEVENTKEY',
'EVENTCODE',
'DISPLAYTRACKINGEVENT',
'EVENTDATE',
'EVENTTIME',
'BOLNUMBER')

In [ ]:
all_return_parcel_tracking_df = total_parcel_events_df.union(missing_delivery_events_parcel)

#### ========================================= Checkpoint 5 =========================================

In [ ]:
all_return_parcel_tracking_df.createOrReplaceTempView('V_all_return_parcel_tracking_df')
session.sql('create table hive_all_return_parcel_tracking_df as select * from V_all_return_parcel_tracking_df')

In [ ]:
all_return_parcel_tracking_df = session.sql('select * from hive_all_return_parcel_tracking_df')

#### ========================================= Checkpoint 5 END =========================================

# BACKFILLING  | Count

In [ ]:
delivery_tracking_label_df.select('TRACKINGLABELKEY').distinct().count()

In [ ]:
missing_delivery_events_parcel.select('TRACKINGLABELKEY').distinct().count()

In [ ]:
all_return_parcel_tracking_df.select('TRACKINGLABELKEY').distinct().count()

In [ ]:
all_return_parcel_tracking_df.count()

In [ ]:
all_return_parcel_tracking_df\
.filter((col('EVENTDATE').isNull()) | (col('EVENTDATE') == 'N'))\
.select('TRACKINGLABELKEY').distinct().count()

# Business FIlteration

In [ ]:
return_parcel_tracking_with_duplicates_events_df=all_return_parcel_tracking_df\
.filter(col('TRACKINGEVENTKEY').isin(['7','13','14','33','326']))\
.select('ID',
'TRACKINGLABELKEY',
'CREATEDATEKEY',
'PRIMARYBARCODE',
'SORTCODE',
'MERCHANTORGKEY',
'DESTFACILITYKEY',
'DESTFACILITYNAME',
'EVENTFACILITYKEY',
'EVENTFACILITYNAME',
'TRACKINGEVENTKEY',
'EVENTCODE',
'DISPLAYTRACKINGEVENT',
'EVENTDATE',
'EVENTTIME',
'BOLNUMBER')

In [ ]:
return_parcel_tracking_df=return_parcel_tracking_with_duplicates_events_df.dropDuplicates(['ID','BOLNUMBER','TRACKINGLABELKEY',\
                                                                    'PRIMARYBARCODE','CREATEDATEKEY','SORTCODE','MERCHANTORGKEY',\
                                                                    'DESTFACILITYKEY','DESTFACILITYNAME',\
                                                                    'EVENTFACILITYKEY','EVENTFACILITYNAME',\
                                                                    'TRACKINGEVENTKEY','EVENTCODE','DISPLAYTRACKINGEVENT'])

#### ========================================= Checkpoint 6 =========================================

In [ ]:
return_parcel_tracking_df.createOrReplaceTempView('V_return_parcel_tracking_df')
session.sql("create table hive_return_parcel_tracking_df as select * from V_return_parcel_tracking_df")

In [ ]:
return_parcel_tracking_df = session.sql('select * from hive_return_parcel_tracking_df')

#### ========================================= Checkpoint 6 Ends =========================================

# BUSINESS FILTERATION | Count

In [ ]:
return_parcel_tracking_with_duplicates_events_df.select('TRACKINGLABELKEY').distinct().count()

In [ ]:
return_parcel_tracking_df.select('TRACKINGLABELKEY').distinct().count()

# Analysis On Final Data

In [ ]:
return_parcel_tracking_df\
.filter(col('DESTFACILITYKEY') == '0').select('TRACKINGLABELKEY').distinct().count()

In [ ]:
return_parcel_tracking_df\
.filter(col('DESTFACILITYKEY').isNull()).select('TRACKINGLABELKEY').distinct().count()

In [ ]:
return_parcel_tracking_df\
.filter(col('DESTFACILITYKEY') == 'N').select('TRACKINGLABELKEY').distinct().count()

In [ ]:
return_parcel_tracking_df\
.filter(col('DESTFACILITYKEY') == '1').select('TRACKINGLABELKEY').distinct().count()

In [ ]:
return_parcel_tracking_df.filter(col('DESTFACILITYNAME').isNull()).select('TRACKINGLABELKEY').distinct().count()

In [ ]:
return_parcel_tracking_df.filter(col('EVENTFACILITYNAME').isNull()).select('TRACKINGLABELKEY').distinct().count()

In [ ]:
return_parcel_tracking_df\
.filter((col('EVENTDATE').isNull()) | (col('EVENTDATE') == 'N'))\
.select('TRACKINGLABELKEY').distinct().count()

# Analysis On AT&T Specific Parcel

In [ ]:
atnt_ctdi_df=return_parcel_tracking_df\
.filter((return_parcel_tracking_df.DESTFACILITYNAME=='CTDI')\
        | ((return_parcel_tracking_df.MERCHANTORGKEY.isin(['366', '469', '3488']) == True)\
            & (return_parcel_tracking_df.DESTFACILITYNAME.isin(['0', '1']))))\
.select('ID',
'TRACKINGLABELKEY',
'PRIMARYBARCODE',
'SORTCODE',
'MERCHANTORGKEY',
'DESTFACILITYKEY',
'DESTFACILITYNAME',
'EVENTFACILITYKEY',
'EVENTFACILITYNAME',
'TRACKINGEVENTKEY',
'EVENTCODE',
'DISPLAYTRACKINGEVENT',
'EVENTDATE',
'EVENTTIME',
'BOLNUMBER')\
.withColumn('DestFacility2', lit('CTDI').cast("string")).distinct()

In [ ]:
atnt_atclefw_df=return_parcel_tracking_df\
.filter((return_parcel_tracking_df.DESTFACILITYNAME=='ATCLE-FW')\
        | ((return_parcel_tracking_df.MERCHANTORGKEY.isin(['507', '514', '803', '892', '2267', '2639', '2752']) == True)\
            & (return_parcel_tracking_df.DESTFACILITYNAME.isin(['0', '1']))))\
.select('ID',
'TRACKINGLABELKEY',
'PRIMARYBARCODE',
'SORTCODE',
'MERCHANTORGKEY',
'DESTFACILITYKEY',
'DESTFACILITYNAME',
'EVENTFACILITYKEY',
'EVENTFACILITYNAME',
'TRACKINGEVENTKEY',
'EVENTCODE',
'DISPLAYTRACKINGEVENT',
'EVENTDATE',
'EVENTTIME',
'BOLNUMBER')\
.withColumn('DestFacility2', lit('ATCLE-FW').cast("string")).distinct()

#### ========================================= Checkpoint 7 =========================================

In [ ]:
atnt_ctdi_df.createOrReplaceTempView('V_atnt_ctdi_df')
session.sql('create table hive_atnt_ctdi_df as select * from V_atnt_ctdi_df')

In [ ]:
atnt_atclefw_df.createOrReplaceTempView('V_atnt_atclefw_df')
session.sql('create table hive_atnt_atclefw_df as select * from V_atnt_atclefw_df')

In [ ]:
atnt_ctdi_df = session.sql('select * from hive_atnt_ctdi_df')

In [ ]:
atnt_atclefw_df = session.sql('select * from hive_atnt_atclefw_df')

#### ========================================= Checkpoint 7 Ends =========================================